In [1]:
print('run')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1555764416095_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
run

# Homework 9: Big Data

MY_UNIQNAME = 'xinyexu'

Teamwork: 

Discussed and cooperated with "Xiaochuan Guo", "Kaiqi Wang"

This homework assignment builds on the in-class work we did with Spark.
You will be using the [Yelp Academic Dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset) and focusing primaily on the text of the reviews (i.e. the reviews.json.gz file).

**We suggest that you work in groups to make a plan to tackle this homework assignment.**

Here are the four questions that comprise the assignment:

1. List the 50 most common non-stopword words that are unique to *positive* reviews.
2. List the 50 most common non-stopword words that are unique to *negative* reviews.
3. What proportion of *positive* reviews contain at least one of the 50 most common non-stopword words that are unique to *positive* reviews.
4. What proportion of *negative* reviews contain at least one of the 50 most common non-stopword words that are unique to *negative* reviews.

As an example, consider the following two reviews:

* Positive: The meal was great, and the service was the best we ever experienced.
* Negative: The meal was awful.  It was the worst thing we ever experienced.

Assume our stopwords are {'the','was','and','the','was','we','it'}

* Positive unique: {'great', 'service', 'best'}

* Negative unique: {'awful', 'worst', 'thing'}

In this example, each unique word occurs just once, so the concept of "top 50" doesn't make sense.  For your data, you'll need to count the number of times each unique word occurs.

Because this is the final homework assignment in this course, we are leaving it up to you to operationalize most of the details.  For example, you will need to determine what constitutes a positive or a negative review.

You will also need to generate a list of stopwords.  Neither spaCy nor NLTK are available on AWS EMR, so you'll need to be creative in how you get a good list of stopwords into Spark.

Finally, you will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the rdd.sample() function in Spark) to work on a reduced size dataset while you're developing your solution.

### REMEMBER TO TERMINATE YOUR AWS CLUSTER(S) WHEN YOU'RE DONE (OR WHEN YOU TAKE A BREAK)!

In [104]:
# This takes about 3 minutes (180s) on a single-node m4.xlarge cluster
df = spark.read.json('s3://umsi-data-science/data/yelp/review.json.gz')

VBox()

In [6]:
df.printSchema()

VBox()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

In [7]:
df.take(3)

VBox()

[Row(business_id=u'0W4lkclzZThpx3V65bVgig', cool=0, date=u'2016-05-28', funny=0, review_id=u'v0i_UHJMo_hPBq9bxWvW4w', stars=5, text=u"Love the staff, love the meat, love the place. Prepare for a long line around lunch or dinner hours. \n\nThey ask you how you want you meat, lean or something maybe, I can't remember. Just say you don't want it too fatty. \n\nGet a half sour pickle and a hot pepper. Hand cut french fries too.", useful=0, user_id=u'bv2nCi5Qv5vroFiqKGopiw'), Row(business_id=u'AEx2SYEUJmTxVVB18LlCwA', cool=0, date=u'2016-05-28', funny=0, review_id=u'vkVSCC7xljjrAI4UGfnKEQ', stars=5, text=u"Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. \n\nStaff was very helpful and friendly.", useful=0, user_id=u'bv2nCi5Qv5vroFiqKGopiw'), Row(business_id=u'VR6GpWIda3SfvPC-lg9H3w', cool=0, date=u'2016-05-28', funny=0, review_id=u'n6QzIUObkYshz4dz2QRJTw', stars=5, t

In [126]:
# Finally, you will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the rdd.sample() function in Spark) to work on a reduced size dataset while you're developing your solution.
# sample(withReplacement, fraction, seed)
sample = df.sample(False, 0.01, 123)

VBox()

In [12]:
sample.take(3)

VBox()

[Row(business_id=u'meXjqyhTNLFmknY39y2sMg', cool=1, date=u'2014-09-11', funny=1, review_id=u'OPZsR2jCG72uoDNjU71DQQ', stars=5, text=u'Solid beers -- Christmas Ale defines my holiday season.  I moved away a long time ago, but whenever I am home, I try to make a stop at Great Lakes.  Great vibe, super chill, stellar food and beer.', useful=1, user_id=u'qYbWTWH5leltA0bzWAOnmA'), Row(business_id=u'CZyXhvt5sfqazfNyeuH3bQ', cool=1, date=u'2014-04-11', funny=2, review_id=u'4rg4MpX7wNywIqywEv0KMw', stars=4, text=u'Ooh but it\'s so DINKY! I\'ve been meaning to try Wellington Coffee out for... oh, a while, so it was with great delight that I accepted an invitation to meet someone for coffee here one morning this week.\n\nI rolled up at the allotted time, 8:30, and there was plenty of room to sit (well, proportionately to the amount of seating in general, which isn\'t a whole bundle because, I repeat, DINKY). And the seat was one of those super-fun stools that has legs but also swivels - a spinni

In [13]:
# stop words
STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
sw = sc.broadcast(STOPWORDS)

VBox()

In [15]:
sample.filter(sample['stars'] >= 4).count()

VBox()

3389

1. List the 50 most common non-stopword words that are unique to *positive* reviews.

### Define our criteria: positive (i.e. 4- or 5-star) reviews; negative: other stars

In [127]:
# use sampling 
posititve = sample.filter(sample['stars'] >= 4.0).select('text')
neagative = sample.filter(sample['stars'] < 4.0).select('text')

VBox()

In [129]:
# method to split into array in a row
import pyspark.sql.functions as f
test_pos = posititve.withColumn('text', f.split(f.col('text'), ' '))
test_neg = neagative.withColumn('text', f.split(f.col('text'), ' '))

VBox()

In [130]:
test_pos.show()

VBox()

+--------------------+
|                text|
+--------------------+
|[Great, place, to...|
|[An, awesome, pla...|
|[I, went, here, l...|
|[Definitely, one,...|
|[Solid, beers, --...|
|[You, never, know...|
|[Completely, runn...|
|[What, is, a, goo...|
|[It's, a, fun, pl...|
|[Date, &, Time:, ...|
|[Je, suis, y, all...|
|[I, love, this, M...|
|[I, went, here, t...|
|[My, Mother's, cl...|
|[I, love, Green.,...|
|[Regular, movie, ...|
|[I, first, visite...|
|[I, belong, to, t...|
|[OMG, -, just, ha...|
|[Service, was, gr...|
+--------------------+
only showing top 20 rows

In [131]:
a_pos = test_pos.collect()

VBox()

In [132]:
a_neg = test_neg.collect()

VBox()

In [133]:
c_pos = list()
for b in a_pos:
    for i in b:
        for j in i:
            t = j.lower()
            if t not in STOPWORDS:
                c_pos.append(t)

VBox()

In [134]:
c_pos[0:20]

VBox()

[u'great', u'place', u'bring', u'dogs!', u"it's", u'really', u'dog', u'place', u'bring', u'humans', u':p\n\nas', u'soon', u'walked', u'in,', u'dog', u'greeted', u'treats!', u'excited', u'explore', u'store...\n\nthere']

In [135]:
# create new dataframe
from pyspark.sql.types import StringType
wordsDF = spark.createDataFrame(c_pos, StringType())

VBox()

In [136]:
wordsDF.show()

VBox()

+---------------+
|          value|
+---------------+
|          great|
|          place|
|          bring|
|          dogs!|
|           it's|
|         really|
|            dog|
|          place|
|          bring|
|         humans|
|         :p

as|
|           soon|
|         walked|
|            in,|
|            dog|
|        greeted|
|        treats!|
|        excited|
|        explore|
|store...

there|
+---------------+
only showing top 20 rows

In [137]:
wordCountsDF = (wordsDF.groupBy('value').count()).sort('count',ascending=False)
wordCountsDF.show(100)

VBox()

+----------+-----+
|     value|count|
+----------+-----+
|          |54929|
|     great|14867|
|     place|13047|
|      good|11182|
|      food|10731|
|      like| 9760|
|       get| 9382|
|      it's| 8777|
|       one| 8687|
|    really| 8249|
|     would| 7808|
|      time| 7586|
|        go| 7379|
|   service| 7315|
|      also| 6968|
|      best| 6663|
|      love| 6557|
|      back| 6477|
|    always| 6379|
|         -| 5765|
|definitely| 5646|
|       got| 5610|
|      i've| 5429|
|      nice| 5248|
|    little| 4970|
|      even| 4928|
|       i'm| 4799|
|     don't| 4572|
|     staff| 4491|
|        us| 4449|
|       try| 4296|
|      came| 4230|
|     first| 4185|
|      come| 4153|
|  friendly| 4140|
| recommend| 4096|
|      made| 4018|
|      went| 4009|
|   ordered| 3930|
|      make| 3911|
|      much| 3701|
|       new| 3644|
|    pretty| 3638|
|     never| 3526|
|     could| 3512|
|      well| 3507|
|         &| 3429|
|   chicken| 3427|
|       it.| 3386|
|     going|

2. List the 50 most common non-stopword words that are unique to *negative* reviews.

In [ ]:
# colelct above: a_neg = test_neg.collect()

In [139]:
c_neg = list()
for b in a_neg:
    for i in b:
        for j in i:
            t = j.lower()
            if t not in STOPWORDS:
                c_neg.append(t)

VBox()

In [140]:
wordsDF_neg = spark.createDataFrame(c_neg, StringType())

VBox()

In [142]:
wordCountsDF_neg = (wordsDF_neg.groupBy('value').count()).sort('count',ascending=False)
wordCountsDF_neg.show(100)

VBox()

+----------+-----+
|     value|count|
+----------+-----+
|          |38721|
|      like| 8324|
|      food| 8231|
|     would| 7902|
|       get| 7772|
|     place| 7175|
|       one| 6891|
|      good| 6182|
|   service| 5566|
|      time| 5484|
|        go| 5328|
|      back| 5295|
|      it's| 5213|
|     don't| 4955|
|      even| 4788|
|    really| 4723|
|       got| 4671|
|    didn't| 4648|
|        us| 4391|
|     never| 4112|
|      came| 3890|
|   ordered| 3838|
|      told| 3661|
|      said| 3659|
|     could| 3617|
|      went| 3501|
|         -| 3434|
|       i'm| 3252|
|     order| 3141|
|     asked| 3133|
|      also| 3019|
|    people| 3000|
|      come| 2986|
|      much| 2916|
|     first| 2839|
|     going| 2787|
|     great| 2743|
|       two| 2725|
|      know| 2643|
|    pretty| 2639|
|       it.| 2619|
|    little| 2610|
|    wasn't| 2586|
|     still| 2563|
|   minutes| 2551|
|      give| 2546|
|   another| 2537|
|      took| 2528|
|      make| 2500|
|      i've|

In [153]:
# find unique top words among pos and neg
values_pos = wordCountsDF.select('value').collect()
values_neg = wordCountsDF_neg.select('value').collect()

VBox()

In [154]:
(unique_pos,unique_neg)=([],[])
n1 = len(values_pos)
for i in range(200):
    if values_pos[i] not in values_neg[0:200]:
        unique_pos.append(values_pos[i])
        if len(unique_pos)==50:
            break
for i in range(200):
    if values_neg[i] not in values_pos[0:200]:
        unique_neg.append(values_neg[i])
        if len(unique_neg)==50:
            break

VBox()

In [157]:
unique_pos

VBox()

[Row(value=u'friendly'), Row(value=u'amazing'), Row(value=u'fresh'), Row(value=u'super'), Row(value=u'delicious'), Row(value=u'happy'), Row(value=u'highly'), Row(value=u'favorite'), Row(value=u'loved'), Row(value=u'excellent'), Row(value=u'vegas'), Row(value=u'perfect'), Row(value=u'awesome'), Row(value=u'prices'), Row(value=u'coming'), Row(value=u'dinner'), Row(value=u'sweet'), Row(value=u'great.'), Row(value=u'top'), Row(value=u'area'), Row(value=u'free'), Row(value=u'fried'), Row(value=u'family'), Row(value=u'everyone'), Row(value=u'burger'), Row(value=u'coffee'), Row(value=u'enjoyed'), Row(value=u'full'), Row(value=u'clean'), Row(value=u'well.'), Row(value=u'selection'), Row(value=u'delicious.'), Row(value=u'absolutely'), Row(value=u'years'), Row(value=u'able'), Row(value=u'places'), Row(value=u'huge'), Row(value=u'large'), Row(value=u'visit'), Row(value=u'breakfast'), Row(value=u'sushi'), Row(value=u'friends'), Row(value=u'done'), Row(value=u'fun'), Row(value=u'usually'), Row(valu

In [158]:
unique_neg

VBox()

[Row(value=u'told'), Row(value=u'asked'), Row(value=u'minutes'), Row(value=u'bad'), Row(value=u'called'), Row(value=u'nothing'), Row(value=u'call'), Row(value=u'left'), Row(value=u'manager'), Row(value=u'server'), Row(value=u'car'), Row(value=u'put'), Row(value=u'pay'), Row(value=u'ask'), Row(value=u'looked'), Row(value=u'maybe'), Row(value=u'someone'), Row(value=u'used'), Row(value=u'front'), Row(value=u'thought'), Row(value=u'worst'), Row(value=u'finally'), Row(value=u'me.'), Row(value=u'hour'), Row(value=u'half'), Row(value=u'back.'), Row(value=u'anything'), Row(value=u'money'), Row(value=u'tell'), Row(value=u'waiting'), Row(value=u'seemed'), Row(value=u'again.'), Row(value=u'let'), Row(value=u'tasted'), Row(value=u'waitress'), Row(value=u"won't"), Row(value=u"couldn't"), Row(value=u'away'), Row(value=u'store'), Row(value=u'good,'), Row(value=u'almost'), Row(value=u'use'), Row(value=u'least'), Row(value=u'trying'), Row(value=u'stars'), Row(value=u'line'), Row(value=u'10'), Row(value

List the 50 most common non-stopword words that are unique to positive reviews.
List the 50 most common non-stopword words that are unique to negative reviews.
are printed as above

3. What proportion of *positive* reviews contain at least one of the 50 most common non-stopword words that are unique to *positive* reviews.

In [178]:
# clean above words by "Find Replace" in office word into pos list and neg list
neg_listDF = ['told', 'asked', 'mintes', 'bad', 'called', 'nothing', 'call', 'left', 'manager', 'server', 'car', 'pt', 'pay', 'ask', 'looked', 'maybe', 'someone', 'sed', 'front', 'thoght', 'worst', 'finally', 'me.', 'hor', 'half', 'back.', 'anything', 'money', 'tell', 'waiting', 'seemed', 'again.', 'let', 'tasted', 'waitress', "won't", "coldn't", 'away', 'store', 'good,', 'almost', 'se', 'least', 'trying', 'stars', 'line', '10', 'hard', 'walked', 'friend']
pos_litDF = ['friendly', 'amazing', 'fresh', 'super', 'delicious', 'happy', 'highly', 'favorite', 'loved', 'excellent', 'vegas', 'perfect', 'awesome', 'prices', 'coming', 'dinner', 'sweet', 'great.', 'top', 'area', 'free', 'fried', 'family', 'everyone', 'burger', 'coffee', 'enjoyed', 'full', 'clean', 'well.', 'selection', 'delicious.', 'absolutely', 'years', 'able', 'places', 'huge', 'large', 'visit', 'breakfast', 'sushi', 'friends', 'done', 'fun', 'usually', 'care', 'thank', 'food,', 'ice', 'far']

VBox()

In [241]:
# count rows with top 50 pos words
count_beg = 0
for row in a_pos:
    for word in list(row)[0]:
        if word in pos_litDF:
            count_beg += 1
            break
    continue

VBox()

4. What proportion of *negative* reviews contain at least one of the 50 most common non-stopword words that are unique to *negative* reviews.

In [242]:
# count rows with top 50 pos words
count_beg_neg = 0
for row in a_neg:
    for word in list(row)[0]:
        if word in neg_listDF:
            count_beg_neg += 1
            break
    continue

VBox()

In [243]:
print(count_beg, count_beg_neg)

VBox()

(27334, 13929)

In [232]:
# count pos rows of text
test_pos.count()

VBox()

34550

In [233]:
# count neg rows of text
test_neg.count()

VBox()

17900

In [255]:
print(float(count_beg) / 34550)
print(float(count_beg_neg) / 17900)

VBox()

0.791143270622
0.778156424581

What proportion of positive reviews contain at least one of the 50 most common non-stopword words that are unique to positive reviews: 0.791143270622
What proportion of negative reviews contain at least one of the 50 most common non-stopword words that are unique to negative reviews: 0.778156424581